## Converting published digital geologic map data into GeMS
#### Notes on converting the generalized layers of the Alaska state geologic map, SIM 3340, from the 'NSA' schema to GeMS

This notebook documents the process of converting the generalized version of the Alaska state geologic map, published in USGS SIM 3340, from the published schema, aka NSA, into GeMS. Note, however that the generalized layers of the map, **AKStategeolpoly_generalized** and **AKStategeolarc_generalized**, stored as feature classes within the geodatabase **AKStategeol.gdb**, were simplified from more richly attributed feature classes and therefore present an easier task of conversion than converting those detailed feature classes.

This notebook also serves as a tutorial for using tools from the GeMS Tools Python toolbox as well as standard Anaconda modules for data science inside a Jupyter Notebook. One advantage to doing this is that  markdown notes are saved alongside tool input parameters and other python code used to explore and clean data during the conversion.

To add this Notebook to an ArcGIS Pro project either
1. On the Insert tab, click the drop-down next to **New Notebook** and browse to this file, or
2. In the Catalog pane, right-click on **Notebooks** on the **Project** tab and browse to this file.

To open the Notebook, find the notebook under **Notebooks** and double-click or right-click and choose **Open Notebook**

This notebook uses a version of the GeMS toolbox which is under development. You can download it from the ```v3.0``` branch of the ```gems-tools-pro``` repo (https://github.com/usgs/gems-tools-pro/tree/v3.0) and use the location of the ```.pyt``` file as the parameter of the ```ImportToolbox``` in section 4.1.

### 1.0 Imports

In [1]:
import sys, os
import arcpy
import inspect
import pandas as pd

### 2.0 Functions

In [2]:
def eval_bool(param):
    '''Return a boolean for various possibilities of boolean-like values'''
    return_bool = False
    if param in [1, True, '1', 'yes', 'Yes', 'true', 'True', 'on', 'On', 'hella']:
        return_bool = True
 
    return return_bool

### 3.0 Explore the original data

#### 3.1  Set variables to layers in the map
Explore the published feature classes. In the case of this Notebook, I am using it inside of an ArcGIS Pro project which also has a map which contains the two feature classes I will be converting. Since everything is contained in the same project, to get a handle on the layers, I only have to refer to them by name, I don't have to worry about paths. The layer names, made from the feature class names in the original database are a little long so I will assign the names to shorter variables. Just make sure that the feature classes in the map do not have any selections or definition filters on them as those will be honored by any geoprocessing we run here. 

In [3]:
gen_lines = "AKStategeolarc_generalized"
gen_polys = "AKStategeolpoly_generalized"

#### 3.2 Properties of the lines 
I will start with the lines. How many are there in this feature class? Let's compare a couple methods to determine this in terms of processing time. First, there is the ArcGIS tool, **Get Count**:

In [32]:
%%time 
# use cell magic command %% time to time the execution in a cell, use % time to time a line
print(arcpy.GetCount_management(gen_lines))

8353
Wall time: 261 ms


And now, using pandas. First, use the ArcGIS tool TableToNumPyArray and then make a pandas dataframe out of that. To get the number of rows in a dataframe use ```df.shape[0]``` or ```len(df.index)```. Both are faster than GetCount.

In [33]:
%%time
arr = arcpy.da.TableToNumPyArray(gen_lines, 'OBJECTID') # only need one field to check the row count
df = pd.DataFrame(arr)
len(df.index)

Wall time: 102 ms


8353

Get a list of the field names. Remember that ```arcpy.ListFields``` returns field objects which have many properties, including ```.name```

In [34]:
field_objects = arcpy.ListFields(gen_lines)
field_names = [f.name for f in field_objects]
print(field_names)

['OBJECTID', 'SHAPE', 'ARC_CODE', 'LINE_TYPE', 'SHAPE_Length']


Let's get a list of the unique pairs of ```ARC_CODE``` and ```LINE_TYPE``` to make sure those are all consistent.
I could use the Frequency tool in ArcGIS, but I think pandas will do a better job.

In [35]:
arr = arcpy.da.TableToNumPyArray(gen_lines, ('ARC_CODE', 'LINE_TYPE'))
df = pd.DataFrame(arr)
uniq = df.groupby(['ARC_CODE', 'LINE_TYPE']).size().reset_index().rename(columns={0:'count'})
pd.options.display.width = 0  # this should tell pandas to expand the columns as necessary to display the entire value,
# otherwise, long LINE_TYPES will likely be truncated with ellipses, but could also use pd.set_option('display.width', 100)
print(uniq)

    ARC_CODE                                          LINE_TYPE  count
0          4                   Normal fault, location certain\r    270
1          5               Normal fault, location approximate\r    282
2          6           Normal fault, location inferred, queried     33
3         10  Thrust fault, location certain, teeth on right...   1706
4         11  Thrust fault, location approximate, teeth on r...    842
5         12  Thrust fault, location inferred, queried, teet...    229
6         14                            Caldera or crater rim\r    123
7         28         Caldera or crater rim, inferred, concealed     33
8         30  Fault, sense of displacement uncertain, locati...    616
9         31  Fault, sense of displacement uncertain, locati...    300
10        32  Fault, sense of displacement uncertain, locati...     43
11        35  High-angle reverse fault, location certain, te...     19
12        36  High-angle reverse fault, location approximate...     49
13    

Some things to note here about the results to keep in mind for later, after the data have been loaded into the GeMS database:
1. There are a few line types that have very few occurrences. Are these possibly mistakes in identification? Is it important to retain these values in a state-wide generalization?
2. There are some values that end in non-printing, end-of-line characters '\r'. These should be removed at some point.
3. Lines marked 'Caldera rim' should be separated into the GeologicLines feature class defined by the GeMS schema. ContactsAndFaults is for lines that model the surface expressions of the 3D planes that separate rock units and participate in a topologic relationship with MapUnitPolys. Lines that model geomorphology and that are not coincident with map unit polygon boundaries are a different concept than contacts and belong in a different feature class.
4. And the big one is that there are no topologic constraints between the lines and the polygons. The metadata for **AKStategeolarc_generalized** says it *"is a subset of AKStategeol_arc, consisting of caldera rims and major faults."* and the generalized polygons were created by a process of merging, rasterization, raster region smoothing, and then vectorization back to polygons. They were not created from the lines in **AKStategeolarc_generalized**. Thus, the lines are not coincident with polygon bounaries and could not be used to rebuild **MapUnitPolys** if there was to be a change. Not having valid topologic relationships between **ContactsAndFaults** and **MapUnitPolys** breaks Level 2 and 3 GeMS compliance. Not sure what to do about this.

As far as conversion to GeMS goes, the ```LINE_TYPE``` values will need to be parsed into the various GeMS fields. We will use the Attribute By Key Values tool for that.

#### 3.3 Properties of the polygons
Again, lets do some simple data exploration to get an idea of what to look out for later in the conversion.

First, a count:

In [4]:
%%time
arr = arcpy.da.TableToNumPyArray(gen_polys, 'OBJECTID') # only need one field to check the row count
df = pd.DataFrame(arr)
len(df.index)

Wall time: 154 ms


15466

A list of the fields:

In [5]:
field_objects = arcpy.ListFields(gen_polys)
field_names = [f.name for f in field_objects]
print(field_names)

['OBJECTID', 'Shape', 'GROUP_ID', 'GROUP_SYMBOL', 'GROUP_LABEL', 'GROUP_LABEL2', 'GROUP_NAME', 'GROUP_AGE', 'Shape_Length', 'Shape_Area']


And the unique groups of values:

In [6]:
fields = ['GROUP_ID', 'GROUP_SYMBOL', 'GROUP_LABEL', 'GROUP_LABEL2', 'GROUP_NAME', 'GROUP_AGE']
arr = arcpy.da.TableToNumPyArray(gen_polys, fields)
df = pd.DataFrame(arr)
uniq = df.groupby(['GROUP_LABEL2', 'GROUP_SYMBOL']).size().reset_index().rename(columns={0:'count'})
pd.set_option('display.max_rows', 220) 
print(uniq)

    GROUP_LABEL2  GROUP_SYMBOL  count
0          CDbrv           303     19
1           CPxt           245     15
2          CPxwg           456     21
3          CPxwn           255    129
4             Ca           164      9
5            Clg           610    123
6          Clgne           533     33
7          Clgtk           533     45
8            Crc           624      6
9           DCbg           924     60
10          DCmt           895      6
11          DCsp            45     97
12         DCwbl           535     60
13          DOgi           147     45
14          DOls           505      4
15          DOsc           505    146
16          DOtu           805      6
17         DPxcn           797     40
18         DPxga           265     13
19         DPxnl           406     13
20         DPxsb           496     81
21         DPxsq            65     96
22          DSsm           714    131
23          DSum           609      2
24           DSv           316     21
25          

218            g             0    413


Again, there are some classifications that only apply to one or a small handful of features, which seems odd. Keep that in mind. As far as converting to GeMS goes, there are fields that will map directly to GeMS fields, eg

| NSA          | GeMS                     |
|--------------|--------------------------|
| GROUP_LABEL2 | MapUnit                  |
| GROUP_LABEL  | Label                    |
| GROUP_SYMBOL | Symbol (maps to wpgcmyk) |
| GROUP_NAME   | Fullname (in DMU table)  |
| GROUP_AGE    | Age (in DMU table)       |

An obvious value missing is ```DataSource```. For this generalized version of the state geologic map, Ric Wilson did not put data sources into the feature classes, nor are there relationship classes with any of the database tables. To get data sources for the ```GROUP```s here, one has to go through the ```genstate``` table which links ```GROUP_ID``` (or ```GROUP_LABEL```) to ```State_label``` which can then be linked through ```AKstategeol_poly``` to a ```SOURCE``` which can be looked up in ```nsarefs```.  (I don't believe there is a normalized lookup table to link these two values). The simplest solution for the GeMS version is to just use 'SIM3340'. Another way would be to try to collate the data sources for all of the NSACLASS units that were grouped together into a group but to do this would require also publishing some of the database tables and defining relationships. If I can build GeMS tools to recognize pipe-delimited ```DataSourceID```s, this could work.

Deciding what put in the ```Description``` field of a GeMS DMU is a similar problem. The published generalized map plate has unit names but no descriptions. The NSA units on the generalized map that are the same as in the database have descriptions in ```genstate```, but those that are comprised of other units and don't have a single description describing the generalized unit.

### 4.0 Transcribe original data to GeMS geodatabase

#### 4.1 Import the GeMS Tools toolbox
The GeMS tools can be called as standalone scripts, but if the Python toolbox is imported instead, it is more like importing a python module where individual tools are referenced as classes, and we get some of the internal validation that is performed before the tool is run.

Use ```arcpy.ImportToolbox``` and provide the full path to the .pyt file (version 3.0 or higher of the ArcGIS Pro version of the toolbox.)

In [36]:
arcpy.ImportToolbox(r"C:\_AAA\gems\gitspace\gems-tools-pro\gems-toolbox\GeMS Tools.pyt")

<module 'gems'>

The alias for the toolbox is ```gems```. The tools are now available by using ```arcpy.<GeMStoolname>_gems```. You may notice this is the same syntax as for many other ```arcpy``` tools such as ```arcpy.TableToTable_conversion``` or ```arcpy.Delete_management```.

#### 4.2 Create an empty GeMS database 
Let's create an empty GeMS database into which the original data will be loaded. We will use the ```CreateDatabase``` tool which is in the ```Create and Edit``` toolset in the GeMS toolbox. You can find help on using the tool by opening the tool parameter form from the Catalog pane and reading the hover tips, by reading the <a href="https://github.com/usgs/gems-tools-pro/wiki/GeMS_ToolsDocumentation#CreateNewDatabase" target="_blank">Wiki entry</a>), or by calling the tool here and not providing any parameters (in this case, ignore the ExecuteError messages block and I don't know why the Usage information is sometimes printed twice).

In [37]:
arcpy.CreateDatabase_gems()

Usage:
       GeMS_CreateDatabase_Arc10.1.py [directory] [geodatabaseName] [coordSystem]
                    [OptionalElements] [#XSections] [AddEditTracking] [AddRepresentations] [AddLTYPE]
       [directory] Name of a directory. Must exist and be writable.
       [geodatabaseName] Name of gdb to be created, with or without .gdb extension.
       [coordSystem] May select an ESRI projection file, import a spatial reference from an existing dataset, or 
          define a new spatial reference system from scratch.
       [OptionalElements] List of optional feature classes to add to GDB, e.g.,
          [OrientationPoints, CartographicLines, RepurposedSymbols]. May be empty.
       [#XSections] is an integer (0, 1, 2, ...) specifying the intended number of
          cross-sections
       [AddEditTracking] Enables edit tracking on all feature classes. Adds fields created_user, 
          created_date, last_edited_user, and last_edited_date. Dates are recorded in database (local) time. 
  

ExecuteError: Failed to execute. Parameters are not valid.
ERROR 000735: Output Workspace: Value is required
ERROR 000735: Name of new geodatabase: Value is required
ERROR 000735: Spatial reference system: Value is required
Failed to execute (CreateDatabase).


Now, prepare the arguments. Note that ```crs``` can be the name of another feature class with coordinates in the spatial reference you want to use. Outside of an ArcGIS Pro you would have to supply the full path to the feature class, but if you are using this code inside of an ArcGIS Pro project and the layer has been added to a map there, just call the name.

Because of the lines marked 'Caldera rim', we'll add a **GeologicLines** feature class. The full list of optional feature classes is:

```'CartographicLines', 'CorrelationOfMapUnits', 'DataSourcePolys', 'FossilPoints', 'GenericPoints', 'GeochronPoints', 'GeologicLines', 'IsoValueLines', 'MapUnitLines', 'MapUnitPoints', 'MapUnitOverlayPolys', 'MiscellaneousMapInformation', 'OrientationPoints', 'OverlayPolys', 'RepurposedSymbols', 'StandardLithology', 'Stations'```

Copy the classes needed from here and put them in the ```optional``` list below.

Because we will be parsing the ```LINE_TYPE``` into a few fields, set the ```addLtype``` parameter to true. When the data are loaded, we'll save ```LINE_TYPE``` values to ```LTYPE``` and then use the Attribute By Key Values tool to parse the values.

In [ ]:
outdir = r"C:\_AAA\mrp\statemap\nsa2gems"
gdb_name = 'Alaska_generalized'
#crs = 'PROJCS["NAD_1983_Alaska_Albers",GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Albers"],PARAMETER["False_Easting",0.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",-154.0],PARAMETER["Standard_Parallel_1",55.0],PARAMETER["Standard_Parallel_2",65.0],PARAMETER["Latitude_Of_Origin",50.0],UNIT["Meter",1.0]]
# for coordinate system you can supply the path to a .prj file (or the text contents of one, above) or the name of another feature class that is in the coordinate system you want to use
crs = "AKStategeolpoly_generalized"
optional = ['GeologicLines']
xsections = 0
addTrack = False
addLtype = True
addConfs = True

And call the tool:

In [ ]:
%%time
gdb_path = os.path.join(outdir, gdb_name+'.gdb')
if arcpy.Exists(gdb_path):
    arcpy.Delete_management(gdb_path)
    
arcpy.CreateDatabase_gems(outdir, gdb_name, crs, optional, xsections, addTrack, addLtype)

#### 4.3 Load original data
When you load data in the **Catalog** pane by right-clicking on a feature class and selecting **Load Data**, it is the <a href="https://pro.arcgis.com/en/pro-app/latest/tool-reference/data-management/append.htm" target="_blank">Append</a> tool for which you fill out a parameter form. In the cell below, I have defined the parameters for using the tool in arcpy. To get these, I first ran the tool after filling out the parameter form and then used the **Copy Python Command** from the **History** pane (right-click on the history item). I then parsed the parameters to the variables below. Again, running some of the tools that are in this Notebook using their parameter forms will be easier than running them from the Notebook, but the point here is to document the process. 

Note: 
1. you can also create a <a href="https://pro.arcgis.com/en/pro-app/latest/arcpy/get-started/mapping-fields.htm" target="_blank">```FieldMappings```</a> object using arcpy,
2. you may have to review some SQL to write a valid expression if you need to filter features. But remember that you can apply definition queries or selections to your input feature class in the map and they will be honored while in place.

In [ ]:
source_fc = gen_lines
target = r"C:\_AAA\mrp\statemap\nsa2gems\Alaska_generalized.gdb\GeologicMap\ContactsAndFaults"
field_mapping_type = "NO_TEST"
# below is what the field mapping parameter looks like when I copy the entire python command generated by the parameter form
# (with inserted line breaks for readability)

# '''
# Type "Type" true true false 254 Text 0 0,First,#;
# IsConcealed "IsConcealed" true true false 1 Text 0 0,First,#;
# LocationConfidenceMeters "LocationConfidenceMeters" true true false 4 Float 0 0,First,#;
# ExistenceConfidence "ExistenceConfidence" true true false 50 Text 0 0,First,#;
# IdentityConfidence "IdentityConfidence" true true false 50 Text 0 0,First,#;
# Label "Label" true true false 50 Text 0 0,First,#;
# Symbol "Symbol" true true false 254 Text 0 0,First,#;
# DataSourceID "DataSourceID" true true false 50 Text 0 0,First,#;
# Notes "Notes" true true false 254 Text 0 0,First,#;
# ContactsAndFaults_ID "ContactsAndFaults_ID" true true false 50 Text 0 0,First,#;
# LTYPE "LTYPE" true true false 200 Text 0 0,First,#,AKStategeolarc_generalized,LINE_TYPE,0,100
# '''

# but I only have to include the field map objects for fields that will store transcribed values.
field_mapping = '''
LTYPE "LTYPE" true true false 200 Text 0 0,First,#,AKStategeolarc_generalized,LINE_TYPE,0,100
'''
subtype = ""
expression = "ARC_CODE NOT IN (14, 28)"

arcpy.management.Append(source_fc, target, field_mapping_type, field_mapping, subtype, expression)

If you make a mistake and want to re-run the tool you can use Truncate Table to remove all features instead of making a new feature class.

In [ ]:
arcpy.management.TruncateTable(target)

Load all of the 'Caldera rim' lines into **GeologicLines**

In [ ]:
source_fc = gen_lines
target = r"C:\_AAA\mrp\statemap\nsa2gems\Alaska_generalized.gdb\GeologicMap\GeologicLines"
field_mapping_type = "NO_TEST"
field_mapping = '''
LTYPE "LTYPE" true true false 200 Text 0 0,First,#,AKStategeolarc_generalized,LINE_TYPE,0,100
'''
subtype = ""
expression = "ARC_CODE IN (14, 28)"

arcpy.management.Append(source_fc, target, field_mapping_type, field_mapping, subtype, expression)

Load the map unit polygons.

In [ ]:
source_fc = gen_polys
target = r"C:\_AAA\mrp\statemap\nsa2gems\Alaska_generalized.gdb\GeologicMap\MapUnitPolys"
field_mapping_type = "NO_TEST"
field_mapping = '''
MapUnit "MapUnit" true true false 10 Text 0 0,First,#,AKStategeolpoly_generalized,GROUP_LABEL,0,10;
Label "Label" true true false 50 Text 0 0,First,#,AKStategeolpoly_generalized,GROUP_LABEL2,0,10;
Symbol "Symbol" true true false 254 Text 0 0,First,#,AKStategeolpoly_generalized,GROUP_SYMBOL,-1,-1
'''
subtype = ""
expression = ""

arcpy.management.Append(source_fc, target, field_mapping_type, field_mapping, subtype, expression)

In [ ]:
arcpy.management.TruncateTable(target)

### 5.0 Use Attribute By Key Values to parse LYTPE
<a href="https://github.com/usgs/gems-tools-pro/wiki/GeMS_ToolsDocumentation#CreateNewDatabase" target="_blank">**Attribute By Key Values**</a> automates the process of selecting features based on a single-value attribute and then running field calculations on the appropriate GeMS fields in order to parse concatenated values. For example, the ```LTYPE``` value ```Normal fault, location certain``` might be translated to:
```
Type = "normal fault”
IsConcealed = "N"
LocationConfidenceMeters = 200
ExistenceConfidence = "certain"
IdentifyConfidence = "certain"
Symbol = "01.01.03"
```

#### 5.1 Build a key-value text file
To use this tool, we have to first create a data dictionary that maps values between a single-value attribute and the appropriate GeMS fields. This is called the "key value" file and an example, **Dig24K_KeyValues.txt**, that you can start with or edit is in the **Resources** folder adjacent to the location of the toolbox file.

Let's revisit the list of unique **LINE_TYPE** values we built in section 3.2. It would be nice write to that to a text file and separate some of the values by the pipe (```|```) delimiter to save some typing.

In [ ]:
# re-use the df variable as a reminder of how to go from an ArcGIS feature class to a pandas dataframe
# although it is not a best-practice to repeatedly read in a data source, it's very inexpensive here
arr = arcpy.da.TableToNumPyArray(gen_lines, ('LINE_TYPE'))
df = pd.DataFrame(arr)

# cast the LINE_TYPE column of values into a pandas Series object and get the unique values
ltypes = pd.Series(df['LINE_TYPE']).unique()
#ltypes

Note that ```ltypes``` is a numpy array data type. Arrays are [computational advantageous](https://learnpython.com/blog/python-array-vs-list/) when the number of values is very large and for numerical calculations. This isn't the case for the string processing we have to do here and python lists are slightly easier to use. We could convert it using ```list(ltypes)```, but this is a good time to get rid of those pesky end-of-line characters so I will use a list comprehension. 

In [ ]:
ltypes = [line.strip() for line in ltypes]
#for l in ltypes: print(l)

In the key_value.txt file we need to build, each feature class that needs values parsed from a "key" field e.g., ```LTYPE```, into dependent GeMS fields, has a section like this: 
```
name of feature class
original type field | GeMS field 1 | GeMS field 2 | GeMS field n
type 1 | value 1 | value 2 | value n
type 2 | value 1 | value 2 | value n
etc.
```

Let's modify the list, adding all of the contents of this section, before we write the list to a file.

In [ ]:
# iterate through the line types and set some values
# we'll review lcm later in map view, but for now use 50, 100, 200 
# for certain, approximate, and inferred
for i, val in enumerate(ltypes):
    gems_type = val.split(',')[0].lower()
    
    if 'concealed' in val.lower():
        is_concealed = 'y'
    else:
        is_concealed = 'n'
    
    if 'approx' in val.lower():
        lcm = '100'
    elif 'inferred' in val.lower():
        lcm = '200'
    else:
        lcm = '50'
    
    if 'queried' in val.lower():
        ex_conf = 'questionable'
        id_conf = 'questionable'
    else:
        ex_conf = 'certain'
        id_conf = 'certain'
        
    val_list = ' | '.join([val, gems_type, is_concealed, lcm, ex_conf, id_conf])
    ltypes[i] = val_list

# finally, add the header lines for this section
# the name of the feature class and the list of dependent fields
ltypes.insert(0,'ContactsAndFaults')
field_header = ['LTYPE', 'Type', 'IsConcealed', 'LocationConfidenceMeters', 'ExistenceConfidence', 'IdentityConfidence']
ltypes.insert(1, ' | '.join(field_header))

Write the list to a file

In [ ]:
# set a path to the file
key_val_file = r"C:\_AAA\mrp\statemap\nsa2gems\nsa2gems_key_values.txt"

# use open(file, 'a') if you are adding to a file
# note that when you use with:, the file is closed after the indented code has run
with open(key_val_file, 'w') as key_val:
    key_val.write("\n".join(ltypes))

Finally, there will probably be a few odd Types to edit. Double-check in a text editor and clean things up. For example, creating a section for GeologicLines was easier to do manually.

In [ ]:
# take a look at the file
with open(key_val_file, 'r') as key_val:
    print(key_val.read())

#### 5.2  Run the tool
Now, set up variables like we did for the **CreateDatabase** tool and call the **Attribute By Key Values** tool. First, call the tool with no parameters to see the ```Usage``` string.

In [ ]:
arcpy.AttributeByKeyValues_gems()

In [ ]:
# key_val_file was set above when we created and edited the file
key_val_file = r"C:\_AAA\mrp\statemap\nsa2gems\nsa2gems_key_values.txt"
gdb = r"C:\_AAA\mrp\statemap\nsa2gems\Alaska_generalized_1.gdb"
# call the tool
arcpy.AttributeByKeyValues_gems(gdb, key_val_file, True)

#### 6.0 Calculate FGDC symbols for lines
Based on the attributes we just calculated using the **Attribute By Key Values** tool and a display scale for the map data, we can calculate the appropriate FGDC symbol using the **Set Symbol Value** tool 

In [ ]:
arcpy.SetSymbols_gems()